In [ ]:
import pandas as pd
import numpy as np

# Carregar os dados
# Arquivo com clustering
clustering_df = pd.read_csv('/Users/marcelosilva/Desktop/clustering(0-4)/4-Clustering/clustering_multiplos_grupos.csv')

# Arquivo com idades
idades_df = pd.read_csv('/Users/marcelosilva/Desktop/clustering(0-4)/1-Sample/EnaniDS.csv')

print("Dados de clustering carregados:")
print(f"Shape: {clustering_df.shape}")
print(clustering_df.head())
print(f"\nColunas: {clustering_df.columns.tolist()}")

print("\n" + "="*50)

print("Dados de idades carregados:")
print(f"Shape: {idades_df.shape}")
print(idades_df.head())
print(f"\nColunas: {idades_df.columns.tolist()}")

# Verificar se a coluna de idade existe
if 'b05a_idade_em_meses' in idades_df.columns:
    print(f"\nColuna 'b05a_idade_em_meses' encontrada!")
    print(f"Valores únicos de idade (primeiros 10): {sorted(idades_df['b05a_idade_em_meses'].dropna().unique())[:10]}")
else:
    print(f"\nColuna 'b05a_idade_em_meses' NÃO encontrada!")
    print(f"Colunas disponíveis que contêm 'idade': {[col for col in idades_df.columns if 'idade' in col.lower()]}")

# Verificar colunas de ID para fazer o merge
print(f"\nColunas no clustering_df: {clustering_df.columns.tolist()}")
print(f"Colunas no idades_df: {idades_df.columns.tolist()}")

# Assumindo que a primeira coluna do clustering é o ID do paciente
id_col_clustering = clustering_df.columns[0]
cluster_col = clustering_df.columns[1] if len(clustering_df.columns) > 1 else clustering_df.columns[1]

print(f"\nUsando coluna ID do clustering: '{id_col_clustering}'")
print(f"Usando coluna cluster: '{cluster_col}'")

# Procurar coluna ID no arquivo de idades (geralmente id_anon ou similar)
possible_id_cols = [col for col in idades_df.columns if 'id' in col.lower() or 'anon' in col.lower()]
print(f"Possíveis colunas de ID no arquivo de idades: {possible_id_cols}")

# Você pode ajustar aqui o nome da coluna ID do arquivo de idades
id_col_idades = possible_id_cols[0] if possible_id_cols else idades_df.columns[0]
print(f"Usando coluna ID das idades: '{id_col_idades}'")

# Fazer o merge dos dados
merged_df = pd.merge(
    clustering_df[[id_col_clustering, cluster_col]], 
    idades_df[[id_col_idades, 'b05a_idade_em_meses']], 
    left_on=id_col_clustering, 
    right_on=id_col_idades, 
    how='inner'
)

print(f"\nDados após merge:")
print(f"Shape: {merged_df.shape}")
print(merged_df.head())

# Função para extrair números da coluna de idade
def extrair_numero_idade(valor):
    """
    Extrai o número da string de idade
    Ex: '22 meses' -> 22, '18' -> 18, nan -> nan
    """
    if pd.isna(valor):
        return np.nan
    
    # Converter para string se não for
    valor_str = str(valor)
    
    # Usar regex para extrair apenas números (incluindo decimais)
    import re
    numeros = re.findall(r'\d+\.?\d*', valor_str)
    
    if numeros:
        return float(numeros[0])  # Pegar o primeiro número encontrado
    else:
        return np.nan

# Aplicar a função para limpar a coluna de idade
print(f"\nExemplos de valores de idade antes da limpeza:")
print(merged_df['b05a_idade_em_meses'].head(10).tolist())

merged_df['idade_numerica'] = merged_df['b05a_idade_em_meses'].apply(extrair_numero_idade)

print(f"\nExemplos de valores após a limpeza:")
print(merged_df[['b05a_idade_em_meses', 'idade_numerica']].head(10))

# Remover valores nulos de idade após limpeza
merged_df_clean = merged_df.dropna(subset=['idade_numerica'])
print(f"\nDados após remover valores nulos de idade:")
print(f"Shape: {merged_df_clean.shape}")

# Verificar se ainda há valores não numéricos
print(f"\nTipo de dados da idade_numerica: {merged_df_clean['idade_numerica'].dtype}")
print(f"Valores únicos de idade (primeiros 20): {sorted(merged_df_clean['idade_numerica'].unique())[:20]}")

# Calcular estatísticas por cluster
estatisticas_por_cluster = merged_df_clean.groupby(cluster_col)['idade_numerica'].agg([
    ('min', 'min'),
    ('max', 'max'), 
    ('mediana', 'median'),
    ('media', 'mean'),
    ('std', 'std'),
    ('count', 'count')
]).round(2)

# Calcular o range (max - min)
estatisticas_por_cluster['range'] = estatisticas_por_cluster['max'] - estatisticas_por_cluster['min']

# Reorganizar colunas
estatisticas_por_cluster = estatisticas_por_cluster[['count', 'min', 'max', 'range', 'mediana', 'media', 'std']]

print("\n" + "="*80)
print("ESTATÍSTICAS DE IDADE (EM MESES) POR CLUSTER:")
print("="*80)
print(estatisticas_por_cluster)

# Criar um resumo mais detalhado
print("\n" + "="*80)
print("RESUMO DETALHADO POR CLUSTER:")
print("="*80)

for cluster in sorted(merged_df_clean[cluster_col].unique()):
    cluster_data = merged_df_clean[merged_df_clean[cluster_col] == cluster]['idade_numerica']
    
    print(f"\nCluster {cluster}:")
    print(f"  • Número de pacientes: {len(cluster_data)}")
    print(f"  • Idade mínima: {cluster_data.min():.1f} meses")
    print(f"  • Idade máxima: {cluster_data.max():.1f} meses") 
    print(f"  • Range: {cluster_data.max() - cluster_data.min():.1f} meses")
    print(f"  • Mediana: {cluster_data.median():.1f} meses")
    print(f"  • Média: {cluster_data.mean():.1f} meses")
    print(f"  • Desvio padrão: {cluster_data.std():.1f} meses")

# Salvar resultados em CSV
output_path = '/Users/marcelosilva/Desktop/clustering(0-4)/estatisticas_idade_por_cluster.csv'
estatisticas_por_cluster.to_csv(output_path)
print(f"\n✅ Resultados salvos em: {output_path}")

# Criar também um arquivo com os dados completos (pacientes + cluster + idade)
output_dados_completos = '/Users/marcelosilva/Desktop/clustering(0-4)/dados_completos_cluster_idade.csv'
# Salvar apenas as colunas importantes
colunas_salvar = [id_col_clustering, cluster_col, 'b05a_idade_em_meses', 'idade_numerica']
merged_df_clean[colunas_salvar].to_csv(output_dados_completos, index=False)
print(f"✅ Dados completos salvos em: {output_dados_completos}")